In [1]:

import pandas as pd
import clickhouse_connect
import os
import missingno as msno
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

# constants
CH_USER = os.getenv("CH_USER")
CH_PASS = os.getenv("CH_PASS")
CH_IP = os.getenv('CH_IP')

# from tools import create_db_table_from_df, pd_tools, spark_tools

root_path = "."
tmp_path = f'{root_path}/tmp'
path_data = f'{root_path}/data'
path_prod_data = f'{path_data}/prod_data'
path_new_data=f'{path_data}/new_data'
path_tmp_data=f'{path_data}/tmp_data'

ch_client = clickhouse_connect.get_client(host=CH_IP, port=8123, username=CH_USER, password=CH_PASS)

In [2]:
messages = pd.read_csv(f'{path_data}/messages.csv')
messages

,message_id,sender_id,receiver_id,reply_message_id
0,1,1,2,NaN
1,3,2,1,1.0
2,19,3,4,16.0
3,16,4,3,1.0
4,1,3,4,NaN
5,4,1,2,3.0
6,7,3,4,NaN
7,17,4,2,7.0


In [6]:
messages['reply_message_id'] = messages['reply_message_id'].astype('Int32')
messages

,message_id,sender_id,receiver_id,reply_message_id
0,1,1,2,<NA>
1,3,2,1,1
2,19,3,4,16
3,16,4,3,1
4,1,3,4,<NA>
5,4,1,2,3
6,7,3,4,<NA>
7,17,4,2,7


In [7]:
# upload data to clickhouse
ch_client.insert_df('messages', messages, 'tmp')

In [9]:
ch_client.query_df('''select * from tmp.messages''')

,message_id,sender_id,receiver_id,reply_message_id
0,1,1,2,<NA>
1,3,2,1,1
2,19,3,4,16
3,16,4,3,1
4,1,3,4,<NA>
5,4,1,2,3
6,7,3,4,<NA>
7,17,4,2,7


Пользователи 1 и 2 общались в рамках одного треда. который можно представить в виде
Цепочки из message id: 1 — 3 — 4. 

Пользователи 3 и 4 общались в рамках двух тредов,которые также можно представить в виде цепочек из message id: 1 —* 16 -+ 19 и 7 — 17.

Вашей задачей является создание запроса на SQLite, который для каждого пользователя
посчитает максимальную длину треда, в котором он принимал участие (длиной треда является
количество сообщений в нем). 

Результатом должна быть таблица с двумя колонками: 
- user_id (идентификатор пользователя из sender_id/receiver_id) 
- max_thread_messages (максимальная длина треда для данного пользователя). 

Выходная таблица должна быть отсортирована по убыванию max_thread messages (в случае равенства мах thread messages
предпочтение должно отдаваться пользователю с наименьшим user_id). 

In [23]:
ch_client.query_df('''
    select 
        t.message_id,
        t.sender_id,
        t.receiver_id,
        t.reply_message_id
    from tmp.messages t
    where 
        t.message_id in (1, 3, 4) 
        and t.sender_id = 1
        and t.receiver_id = 1
''')

""
